In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sales = pd.read_csv('kc_house_data.csv')

In [3]:
sales.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':float, 'condition':int, 'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}


In [5]:
sales = pd.read_csv('kc_house_data.csv', dtype= dtype_dict)
sales.dtypes

id                object
date              object
price            float64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode           object
lat              float64
long             float64
sqft_living15    float64
sqft_lot15       float64
dtype: object

In [6]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].map(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms'].map(lambda x:x**2)
sales['floors_square'] = sales['floors'].map(lambda x:x**2)


In [7]:
def transformation(data):
    data['sqft_living_sqrt'] = data['sqft_living'].map(sqrt)
    data['sqft_lot_sqrt'] = data['sqft_lot'].apply(sqrt)
    data['bedrooms_square'] = data['bedrooms'].map(lambda x:x**2)
    data['floors_square'] = data['floors'].map(lambda x:x**2)


In [8]:
cols_trasformed_cols = [col for col in sales.columns if(col.startswith('sqft') or col.startswith('bedrooms'))]
cols_trasformed_cols

['bedrooms',
 'sqft_living',
 'sqft_lot',
 'sqft_above',
 'sqft_basement',
 'sqft_living15',
 'sqft_lot15',
 'sqft_living_sqrt',
 'sqft_lot_sqrt',
 'bedrooms_square']

In [9]:
sales[['bedrooms_square',
 'bedrooms',      
'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'sqft_above',
 'sqft_basement',
 'sqft_living15',
 'sqft_lot15']].head()

,bedrooms_square,bedrooms,sqft_living,sqft_living_sqrt,sqft_lot,sqft_lot_sqrt,sqft_above,sqft_basement,sqft_living15,sqft_lot15
0,9.0,3.0,1180.0,34.351128,5650,75.166482,1180,0,1340.0,5650.0
1,9.0,3.0,2570.0,50.695167,7242,85.099941,2170,400,1690.0,7639.0
2,4.0,2.0,770.0,27.748874,10000,100.000000,770,0,2720.0,8062.0
3,16.0,4.0,1960.0,44.271887,5000,70.710678,1050,910,1360.0,5000.0
4,9.0,3.0,1680.0,40.987803,8080,89.888820,1680,0,1800.0,7503.0


In [10]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

## Quiz Question: Which features have been chosen by LASSO, i.e. which features were assigned nonzero weights?

In [11]:
len(model_all.coef_)

17

In [12]:
len(all_features)

17

In [13]:
[all_features[i] for i in range(len(all_features)) if(model_all.coef_[i]!=0.)]

['sqft_living', 'view', 'grade']

In [14]:
#Features with 0 weights
[all_features[i] for i in range(len(all_features)) if(model_all.coef_[i]==0.)]

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'condition',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

## Splitting the data into Training, Testing and Validation Sets

In [15]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [16]:
list(map(lambda x:'price' in x.columns, [testing, training, validation]))

[True, True, True]

In [17]:
#Transform the respective features of the 3 datasets
list(map(transformation, [testing, training, validation]))

[None, None, None]

In [18]:
import sys
m = sys.float_info.max

penalty_smallest_rss = ''

for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    
    #Fit the model on training data
    model.fit(training[all_features], training['price'])
    
    #Predict on validation data
    predicted_val = model.predict(validation[all_features])
    
    #Compute the RSS on VALIDATION for the current model 
    rss_val = np.sum(np.array([(pred-act)**2 for pred, act in zip(predicted_val, validation['price'])]))
    
    #RSS_for_models.append(rss_val)
    if(rss_val <= m):
        m = rss_val
        penalty_smallest_rss = l1_penalty
        
print(penalty_smallest_rss)

10.0


## Quiz Question: Which was the best value for the l1_penalty, i.e. which value of l1_penalty produced the lowest RSS on VALIDATION data?

In [19]:
10.0

10.0

## Quiz Question: Using the best L1 penalty, how many nonzero weights do you have? Count the number of nonzero coefficients first, and add 1 if the intercept is also nonzero.

In [20]:
#compute the RSS on TEST data for the model with the best L1 penalty.

best_l1_penalty = 10.0
model = linear_model.Lasso(alpha=best_l1_penalty, normalize=True)

#Fit the model on training data
model.fit(training[all_features], training['price'])

#Predict on test data
predicted_test = model.predict(testing[all_features])

#compute the RSS on TEST data
rss_test = np.sum(np.array([(pred-act)**2 for pred, act in zip(predicted_test, testing['price'])]))

print("RSS on Test data:- {}" .format(rss_test)) 

RSS on Test data:- 98467402552698.8


In [21]:
#How many non-zero weights do we have
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

In [22]:
model.coef_

array([-1.61445628e+04,  3.73245384e+02,  5.08412433e+04,  6.17853560e+02,
       -4.44113549e+04,  7.85623065e-01, -7.01194765e+02, -0.00000000e+00,
        5.01420046e+03,  6.19488752e+05,  3.80418557e+04,  2.49987718e+04,
        1.28716235e+05,  0.00000000e+00,  0.00000000e+00, -3.29383118e+03,
        1.00573209e+01])

In [23]:
model.intercept_

6630155.668628362

### What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

Explore a large range of ‘l1_penalty’ values to find a narrow region of ‘l1_penalty’ values where models are likely to have the desired number of non-zero weights.

Further explore the narrow region you found to find a good value for ‘l1_penalty’ that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for ‘l1_penalty’.

In [24]:
# Assign 7 to the variable ‘max_nonzeros’.
max_nonzeros = 7

In [32]:
max(np.logspace(1, 4, num=20))

10000.0

In [33]:
1 + (20-1)*4 + 1

78

In [34]:
np.logspace(1, 4, num=20)

array([   10.        ,    14.38449888,    20.69138081,    29.76351442,
          42.81332399,    61.58482111,    88.58667904,   127.42749857,
         183.29807108,   263.66508987,   379.26901907,   545.55947812,
         784.75997035,  1128.83789168,  1623.77673919,  2335.72146909,
        3359.81828628,  4832.93023857,  6951.92796178, 10000.        ])

In [37]:

l1_penalty_min = max(np.logspace(1, 4, num=20))+1
l1_penalty_max = 0
non_zero_list = []

#Exploring large range of l1_penalty
for l1_penalty in np.logspace(1, 4, num=20):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)

    #Fit the model on the train data
    model.fit(training[all_features], training['price'])
    
    #Extract the weights of the model and count the number of nonzeros
    count = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    non_zero_list.append(count)
    
    #The largest l1_penalty that has more non-zeros than ‘max_nonzeros’
    if(count>max_nonzeros):
        if(l1_penalty > l1_penalty_max):
            l1_penalty_max = l1_penalty
    
    #The smallest l1_penalty that has fewer non-zeros than ‘max_nonzeros’
    if(count<max_nonzeros):
        if(l1_penalty < l1_penalty_min):
            l1_penalty_min = l1_penalty
    

### Quiz Question: What values did you find for l1_penalty_min and l1_penalty_max?

In [38]:
print("Max L1 penalty= %.2f\nMin L1 penalty= %.2f" %(l1_penalty_max, l1_penalty_min))

Max L1 penalty= 127.43
Min L1 penalty= 263.67


## Exploring narrower range of l1_penalty

In [39]:
np.linspace(l1_penalty_min,l1_penalty_max,20)

array([263.66508987, 256.49469033, 249.32429079, 242.15389125,
       234.9834917 , 227.81309216, 220.64269262, 213.47229308,
       206.30189354, 199.13149399, 191.96109445, 184.79069491,
       177.62029537, 170.44989582, 163.27949628, 156.10909674,
       148.9386972 , 141.76829765, 134.59789811, 127.42749857])

In [44]:
#Find the lowest RSS on validation data
m = sys.float_info.max
lowest_rss = ''
l1_penalty_lowest_rss = ''

for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    #Create a model with the respective l1_penalty
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    
    #Fit the model on training data
    model.fit(training[all_features], training['price'])
    
    #Predict on the validation set
    predicted_val = model.predict(validation[all_features])
    
    #Calculate the RSS
    rss_val = np.sum(np.array([(act-pred)**2 for act, pred in zip(validation['price'], predicted_val)]))
    #print(np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))
    if(rss_val<m):
        
        if((np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_))==max_nonzeros):
            lowest_rss = rss_val
            m = rss_val
            l1_penalty_lowest_rss = l1_penalty
            print("RSS:- %.2f\tL1 penalty:- %.2f" %(rss_val, l1_penalty))

RSS:- 445230739842614.19	L1 penalty:- 199.13
RSS:- 444239780526141.62	L1 penalty:- 191.96
RSS:- 443296716874315.06	L1 penalty:- 184.79
RSS:- 442406413188666.25	L1 penalty:- 177.62
RSS:- 441566698090139.88	L1 penalty:- 170.45
RSS:- 440777489641605.19	L1 penalty:- 163.28
RSS:- 440037365263316.50	L1 penalty:- 156.11


In [45]:
l1_penalty_lowest_rss

156.10909673930752

In [46]:
lowest_rss

440037365263316.5

## Consider the model learned with the l1_penalty found in the previous question. Which of the following features has non-zero coefficients? 

In [47]:
model_lowest_l1_penalty = linear_model.Lasso(alpha=l1_penalty_lowest_rss, normalize=True)

#Fit the model on training data
model_lowest_l1_penalty.fit(training[all_features], training['price'])

#Features having non-zero coefficients
[all_features[i] for i in range(len(all_features)) if(model_lowest_l1_penalty.coef_[i]!=0.)]

['bathrooms', 'sqft_living', 'waterfront', 'view', 'grade', 'yr_built']